In [1]:
import geetools
import ee
import geemap
import random
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import math
import os
from datetime import datetime
import skimage
import time
from A1_landsat_functions import *


In [2]:
ee.Authenticate(auth_mode="notebook")

True

In [3]:
ee.Initialize()

In [5]:
#define coordinates to use 
coords_jd = (39.0053612, 21.4210088, 39.3253612, 21.7410088)  # Jeddah
coords_md = (39.4511216, 24.311153, 39.7711216, 24.631153)   # Medina
coords_mk = (39.698,21.078,40.222,21.694)     # Mecca
coords_tf = (40.2558308, 21.1102801, 40.5758308, 21.4302801)   # Taif
#coords_rd 
coords_rd = (46.5560104, 24.478916, 46.8760104, 24.798916) #Riyadh bbox
coord_rd = [[46.4328690955,24.1659202907],[46.6345003059,25.4285645773],[47.5182109888,25.2818982646],
             [47.5209967609,24.1956804525],[46.4328690955,24.1659202907]] #riyad polygon

coords = coords_jd

buffered_coords = add_degree_buffer_to_coords(coords, 0.2)
bbox = ee.Geometry.BBox(*buffered_coords) 
#bbox = ee.Geometry.Polygon([coord_rd])

In [4]:
#define panels to use 
#Makka
#WRS_PATH = 169
#WRS_ROW = 45
#Jeddah
WRS_PATH = 170
WRS_ROW = 45
#riyadh
#WRS_PATH = 165
#WRS_ROW = 43

In [6]:
#get Data: Landsat 
# Landsat collections (T1 = Tier 1 Surface Reflectance where available)
landsat1 = ee.ImageCollection("LANDSAT/LM01/C02/T1")
landsat2 = ee.ImageCollection("LANDSAT/LM02/C02/T1")
landsat3 = ee.ImageCollection("LANDSAT/LM03/C02/T1")
landsat4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")
landsat5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
landsat7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

# Merge Landsat collections based on sensors
landsat_1_3 = landsat1.merge(landsat2).merge(landsat3).filterBounds(bbox).sort("DATE_ACQUIRED")
landsat_4_7 = landsat4.merge(landsat5).merge(landsat7).filterBounds(bbox).sort("DATE_ACQUIRED")
landsat8 = landsat8.filterBounds(bbox).sort("DATE_ACQUIRED")

landsat_1_3 = landsat_1_3.filter(ee.Filter.And(ee.Filter.eq('WRS_PATH', WRS_PATH),ee.Filter.eq('WRS_ROW', WRS_ROW))).map(lambda img: img.clip(bbox))
landsat_4_7 = landsat_4_7.filter(ee.Filter.And(ee.Filter.eq('WRS_PATH', WRS_PATH),ee.Filter.eq('WRS_ROW', WRS_ROW))).map(lambda img: img.clip(bbox))
landsat8    = landsat8.filter(ee.Filter.And(ee.Filter.eq('WRS_PATH', WRS_PATH),ee.Filter.eq('WRS_ROW', WRS_ROW))).map(lambda img: img.clip(bbox))

#apply processing for cloud mask and rename bands
landsat_4_7 = landsat_4_7.map(maskL457sr).map(rescaleL457sr).map(renameL457)
landsat8 = landsat8.map(maskL8sr).map(rescaleL8sr).map(renameL8sr)
#merge landsat data sets 
landsat = landsat_4_7.merge(landsat8)

#apply diffent indices 
landsat_indexed = landsat.map(apply_indices)

#Add LST
landsat_indexed = landsat_indexed.map(add_lst)

In [7]:
exclude_ids = ['1_1_2_LT05_170045_19871122', '1_1_2_LT05_170045_19881226', '2_LC08_170045_20160801', '1_1_2_LT05_169045_19960312', 
               '2_LC08_169045_20200720', '1_1_2_LT05_169045_19840903', '2_LC08_169045_20220115', '1_2_LE07_165043_20190511', 
               '1_1_2_LT05_170045_19970202', '1_2_LE07_170045_20230410']
landsat_indexed = landsat_indexed.filter(ee.Filter.inList('system:index', exclude_ids).Not())


In [ ]:

# Set year range
start_year = 1984
end_year = 1987 #2024

# Sleep interval in seconds
sleep_seconds = 10

# Define image export function
def export_image(img, index):
    try:
        date_str = ee.Date(img.get("system:time_start")).format("YYYYMMdd").getInfo()
        task = ee.batch.Export.image.toDrive(
            image=img.clip(bbox),
            description=f"LST_Jeddah_{date_str}_{index}",
            folder="GEE_Landsat",
            fileNamePrefix=f"Jeddah_LST_{date_str}",
            region=bbox,
            scale=30,
            maxPixels=1e13
        )
        task.start()
        print(f"Started export for {date_str} (Index {index})")
        time.sleep(sleep_seconds)
    except Exception as e:
        print(f"Failed to export image at index {index}: {e}")
        time.sleep(sleep_seconds)

# Export all images year by year
for year in range(start_year, end_year + 1):
    start = ee.Date(f"{year}-01-01")
    end = ee.Date(f"{year}-12-31")
    yearly_collection = landsat_indexed.filterDate(start, end)
    image_list = yearly_collection.toList(yearly_collection.size())
    num_images = image_list.size().getInfo()
    
    print(f"Processing year {year} with {num_images} images")

    for i in range(num_images):
        img = ee.Image(image_list.get(i))
        export_image(img, index=i)


Processing year 1984 with 7 images
Started export for 19840606 (Index 0)
Started export for 19840622 (Index 1)
